In [ ]:
pip install konlpy

In [ ]:
pip install tensorflow

In [28]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import re
import urllib.request
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from konlpy.tag import Okt

In [4]:
from sklearn.model_selection import train_test_split

In [8]:
!ls

210223_dataset1.csv  sample_data


In [15]:
# 파일 확장자에 맞춰서 불러오기. csv -> pd.read_csv
train_data = pd.read_csv('210223_dataset1.csv', encoding='cp949')

In [17]:
train, test= train_test_split(train_data, test_size=0.3, random_state = 34)

In [18]:
train

,제목,상승/하락
9548,코로나에도 실적 선방,1.0
6557,올해 1순위 청약통장 '자이'에 가장 많이 몰렸다,1.0
14184,피라맥스 코로나19 치료제 임상 승인 기대감에 강세,1.0
11118,·제약 운명의 날 내달로 연기…7월6일 결착,1.0
2007,·YG 네이버 AR 서비스에 120억 투자,1.0
...,...,...
12022,코로나19 백신 소식에 12% 급락…실적 좋지만 악재 대비해야,0.0
5993,개인 투자자 관심 집중... 주가 1.55%...,1.0
10730,제약 ‘올로스타’ 개별약제 병용요법 대비 우수한 효과 확인,0.0
5242,현대· 3년간 산재보험급여 500억 넘게 써,0.0


In [20]:
print('훈련용 리뷰 개수 :',len(train)) # 훈련용 리뷰 개수 확인

훈련용 리뷰 개수 : 9947


In [21]:
print('테스트용 리뷰 개수 :',len(test)) # 테스트용 리뷰 개수 확인

테스트용 리뷰 개수 : 4264


In [22]:
test[:5] #데이터 형태 확인을 위한 상위 5개 데이터 출력

,제목,상승/하락
3700,"애플카, 현대 전기차 플랫폼 활용…이르면 2025년 생산",0.0
169,카카오엔터 전략적 업무협약…해운물류산업에 AI 이식한다,1.0
5040,"테슬라·아우디보다 위···현대차, 글로벌 자동차 브랜드 톱5 최초 진입",1.0
6752,국내 첫 지하주차장 외벽 PC공법 확대 적용,1.0
6108,과 이미지 통합… 신규 CI 발표,0.0


In [26]:
# 한글과 공백을 제외하고 모두 제거
train['제목'] = train['제목'].str.replace("[^ㄱ-ㅎ ㅏ-ㅣ 가-힣 a-z A-Z]","") 
test['제목'] = test['제목'].str.replace("[^ㄱ-ㅎ ㅏ-ㅣ 가-힣 a-z A-Z]","") 
#영어까지 제외 하고 전부 제거
train[:5]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,제목,상승/하락
9548,코로나에도 실적 선방,1.0
6557,올해 순위 청약통장 자이에 가장 많이 몰렸다,1.0
14184,피라맥스 코로나 치료제 임상 승인 기대감에 강세,1.0
11118,제약 운명의 날 내달로 연기월일 결착,1.0
2007,YG 네이버 AR 서비스에 억 투자,1.0


In [30]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다',
             '현대차', '로', '기','현대','한전', '기아차','삼성','억','등','년','만','천','적','억원','전','비','일','원','제','위','상'
             '서','대','코로나','분기','조','로','이','주'] #불용어

X_train = []
for sentence in train['제목']:
    temp = []
    temp = Okt().morphs(sentence, stem=True) # 토큰화
    temp = [word for word in temp if not word in stopwords] # 불용어 제거
    X_train.append(temp)
X_test = []
for sentence in test['제목']:
    temp = []
    temp = Okt().morphs(sentence, stem=True)
    temp = [word for word in temp if not word in stopwords] 
    X_test.append(temp)

In [31]:
print(X_train[:3]) # 되었는지 3개만 출력해 봄

[['에도', '실적', '선방'], ['올해', '순위', '청약', '통장', '자다', '가장', '많이', '몰리다'], ['피', '라', '맥스', '치료', '임', '상', '승인', '대감', '강세']]


정수 인코딩

In [32]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

# X_train에 있는 것들 토큰화

In [33]:
print(tokenizer.word_index)
# 자주 나오는 순으로 1번이 가장 많이 나옴. 숫자가 클수록 자주 나오지 않는 자연어.

{'주가': 1, '목표': 2, '제약': 3, '실적': 4, '영업': 5, '상승': 6, '그룹': 7, '사업': 8, '투자': 9, '바이오': 10, '판매': 11, '이상': 12, '진단': 13, '규모': 14, '상': 15, '파다': 16, '매출': 17, '키트': 18, '서': 19, '아시아나': 20, '최대': 21, '첫': 22, '출시': 23, '월': 24, '시장': 25, '익': 26, '강세': 27, '전기차': 28, '인수': 29, '기업': 30, '까지': 31, '올해': 32, '시': 33, '매수': 34, '개발': 35, '전망': 36, '배터리': 37, '이익': 38, '급등': 39, '되다': 40, '선': 41, '계약': 42, '만원': 43, '수': 44, '순': 45, '공급': 46, '확대': 47, '중': 48, '글로벌': 49, '장': 50, '화': 51, '치료': 52, '수출': 53, '서비스': 54, '신': 55, '오다': 56, 'q': 57, '지': 58, '에도': 59, '총': 60, '기술': 61, '증가': 62, '성장': 63, '신고': 64, '외국인': 65, '것': 66, '개': 67, '애플': 68, '수주': 69, '지원': 70, '대다': 71, '전환': 72, '대비': 73, '기대': 74, '만에': 75, '카': 76, '개선': 77, '협력': 78, '임': 79, '컨콜': 80, '반도체': 81, '효과': 82, '사': 83, '상장': 84, '돌파': 85, '기관': 86, '없다': 87, '미래': 88, '국내': 89, '성': 90, '연속': 91, '하락': 92, '생명': 93, '차': 94, '내년': 95, '추진': 96, '체결': 97, '증권': 98, '사장': 99, '분양': 100, '케미칼': 101, '흑자': 102

In [34]:
total = len(tokenizer.word_index) #단어의 수
rare = 0
for key, value in tokenizer.word_counts.items():
  if value < 3:
    rare += 1
size = total - rare + 2
print(total)
print(rare)
print(size)

8521
4744
3779


In [35]:
tokenizer = Tokenizer(size, oov_token = 'OOV') 
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [36]:
y_train = np.array(train['상승/하락'])
y_test = np.array(test['상승/하락'])

In [37]:
drop_train = [index for index, sentence in enumerate(X_train) if len(sentence) < 1]

In [38]:
# 빈 샘플들을 제거
X_train = np.delete(X_train, drop_train, axis=0)
y_train = np.delete(y_train, drop_train, axis=0)
print(len(X_train))
print(len(y_train))

9940
9940


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [41]:
X_train = pad_sequences(X_train, maxlen = 30)
X_test = pad_sequences(X_test, maxlen = 30)

# 길이 맞춰줌.

LSTM 학습

In [42]:
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [44]:
model = Sequential()
model.add(Embedding(size, 100))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))

In [45]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)


In [46]:
model.compile(
    optimizer='rmsprop', 
    loss='binary_crossentropy', 
    metrics=['acc'])


history = model.fit(
    X_train, 
    y_train, 
    epochs=15, 
    callbacks=[es, mc], 
    batch_size=60, 
    validation_split=0.2
    )
# 훈련데이터 20퍼를 검증 데이터로 사용 함. (0.2)

Epoch 1/15
133/133 [==============================] - 11s 66ms/step - loss: 0.6921 - acc: 0.5143 - val_loss: 0.6828 - val_acc: 0.5458

Epoch 00001: val_acc improved from -inf to 0.54577, saving model to best_model.h5
Epoch 2/15
133/133 [==============================] - 8s 60ms/step - loss: 0.6394 - acc: 0.6478 - val_loss: 0.7300 - val_acc: 0.5553

Epoch 00002: val_acc improved from 0.54577 to 0.55533, saving model to best_model.h5
Epoch 3/15
133/133 [==============================] - 8s 62ms/step - loss: 0.5780 - acc: 0.6980 - val_loss: 0.7080 - val_acc: 0.5644

Epoch 00003: val_acc improved from 0.55533 to 0.56439, saving model to best_model.h5
Epoch 4/15
133/133 [==============================] - 8s 59ms/step - loss: 0.5281 - acc: 0.7412 - val_loss: 0.7384 - val_acc: 0.5649

Epoch 00004: val_acc improved from 0.56439 to 0.56489, saving model to best_model.h5
Epoch 5/15
133/133 [==============================] - 8s 59ms/step - loss: 0.5148 - acc: 0.7574 - val_loss: 0.7713 - val_acc: 

In [47]:
loaded_model = load_model('best_model.h5')
print(loaded_model.evaluate(X_test, y_test)[1])

134/134 [==============================] - 2s 8ms/step - loss: 0.7195 - acc: 0.5671
0.5670731663703918


In [48]:
pre_model = model.predict(X_test)

In [ ]:
print(pre_model) 

In [64]:
def getpredict(s):    
    new_sentence = Okt().morphs(s, stem=True) # 토큰화
    new_sentence = [word for word in s if not word in stopwords] # 불용어 제거
    encoded = tokenizer.texts_to_sequences([s]) # 정수 인코딩
    pad_new = pad_sequences(encoded, maxlen = 30) # 패딩
    score = float(loaded_model.predict(pad_new)) # 예측
    if(score > 0.5):
        return ("{:.2f}% 확률로 호재입니다.\n".format(score * 100))
    else:
        return ("{:.2f}% 확률로 악재입니다.\n".format((1 - score) * 100))

In [54]:
getpredict('대웅제약 이지에프외용액 할랄 인증 획득조 규모 중동시장 진출')

50.54% 확률로 호재입니다.



In [55]:
pip install python-telegram-bot

     |████████████████████████████████| 440kB 14.0MB/s 
     |████████████████████████████████| 61kB 5.9MB/s 


In [ ]:
#실시간 새글알림
import requests
from bs4 import BeautifulSoup as bs
import time
import telegram
my_token = '1692612907:AAH1oqA5-od1ENDi1mwS9rkGbxvk5HoO0yg'
bot = telegram.Bot(token = my_token)
chat_id = '1497406622'
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.104 Safari/537.36'}
temp = []
while True:
  res = requests.get('https://finance.naver.com/news/news_list.nhn?mode=LSS2D&section_id=101&section_id2=258')
  soup = bs(res.content, 'lxml')
  now_title = soup.select_one('dd > a ')['title']
  if now_title.strip() not in temp and len(now_title) > 3:
    temp.append(now_title.strip())
    temp1 = f'{now_title}\n{getpredict(now_title)}'
    bot.sendMessage(chat_id = chat_id, text = temp1)
  time.sleep(5)